In [1]:
import pandas as pd
from tqdm import tqdm
from custom import extract_key_from_json

## Load Data

In [2]:
CHANNEL_TITLE = input("Filename (Should match the channel title): ")

Filename (Should match the channel title):  Dolphine 254


In [3]:
df = pd.read_csv(f"./data/{CHANNEL_TITLE}.csv")

In [4]:
df.shape

(444, 7)

## API Call

In [5]:
import os
from googleapiclient.discovery import build

In [6]:
API_KEY = os.getenv("YOUTUBE_API_KEY")

In [7]:
youtube = build("youtube", "v3", developerKey=API_KEY)

### Video Stats

In [8]:
def get_stats(VIDEO_ID):
    response = youtube.videos().list(
        part="snippet, statistics, contentDetails",
        id=VIDEO_ID
    ).execute()
    return response

In [9]:
video_ids = df.id

In [10]:
video_stats = []
for video_id in tqdm(video_ids):
    video_stats.append(get_stats(video_id))

100%|████████████████████████████████████████████████████████████████████████████████| 444/444 [01:13<00:00,  6.07it/s]


In [12]:
stats = pd.DataFrame(extract_key_from_json(video_stats, 'statistics'))
stats['duration'] = extract_key_from_json(video_stats, 'duration')
stats['date_published'] = extract_key_from_json(video_stats, "publishedAt")

stats.head()

,viewCount,likeCount,favoriteCount,commentCount,duration,date_published
0,2343,123,0,38,PT10M31S,2023-08-31T17:05:14Z
1,1233,50,0,11,PT16M32S,2023-08-28T16:42:20Z
2,6139,243,0,53,PT19M11S,2023-08-26T17:53:13Z
3,3672,119,0,33,PT16M33S,2023-08-19T16:41:20Z
4,1796,105,0,36,PT11M29S,2023-08-16T17:02:53Z


## Merge and Name Dataframe

In [13]:
stats_ = pd.concat([df, stats], axis=1)

In [14]:
VIDEO_ID = df.id[0]

In [15]:
video_response = youtube.videos().list(
    part="snippet",
    id=VIDEO_ID
).execute()

In [16]:
CHANNEL_TITLE = extract_key_from_json(video_response, 'channelTitle')[0]
CHANNEL_TITLE

'Dolphine 254'

## Save

In [17]:
stats_.to_csv(f"./data/{CHANNEL_TITLE}.csv", index=False)